In [ ]:
import os, sys
root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, root)
from src import preprocess
preprocess.main()

Loading raw data from c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw
Found 78 subjects in c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw
Processing subject: sub-01. file_path: c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw
Loading raw data from c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw\sub-01\eeg\sub-01_task-sternberg_eeg.vhdr
Extracting parameters from c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw\sub-01\eeg\sub-01_task-sternberg_eeg.vhdr...
Setting channel info structure...
Processing subject: sub-02. file_path: c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw
Loading raw data from c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw\sub-02\eeg\sub-02_task-sternberg_eeg.vhdr
Extracting parameters from c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw\sub-02\eeg

In [ ]:
from src import preprocess, training, diffe
import importlib

importlib.reload(preprocess)
importlib.reload(training)
importlib.reload(diffe)

<module 'src.diffe' from 'c:\\magister\\probabilistyczne-modele-grafowe\\lab\\mind-reader\\src\\diffe.py'>

In [ ]:
preprocess.main()

Loading raw data from c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw
Found 78 subjects in c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw
Processing subject: sub-01. file_path: c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw
Loading raw data from c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw\sub-01\eeg\sub-01_task-sternberg_eeg.vhdr
Extracting parameters from c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw\sub-01\eeg\sub-01_task-sternberg_eeg.vhdr...
Setting channel info structure...


Processing subject: sub-02. file_path: c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw
Loading raw data from c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw\sub-02\eeg\sub-02_task-sternberg_eeg.vhdr
Extracting parameters from c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw\sub-02\eeg\sub-02_task-sternberg_eeg.vhdr...
Setting channel info structure...
Processing subject: sub-03. file_path: c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw
Loading raw data from c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw\sub-03\eeg\sub-03_task-sternberg_eeg.vhdr
Extracting parameters from c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw\sub-03\eeg\sub-03_task-sternberg_eeg.vhdr...
Setting channel info structure...
Processing subject: sub-04. file_path: c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\raw
Loading raw

In [ ]:
train_ds, val_ds, test_ds = training.get_datasets(use_zscore=True)

c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\training.py:69: RuntimeWarning: This filename (c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\preprocessed\sub-01_preprocessed_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  datasets.append(EpochsDataset(epochs))
c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\training.py:69: RuntimeWarning: This filename (c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\preprocessed\sub-02_preprocessed_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  datasets.append(EpochsDataset(epochs))
c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\training.py:69: RuntimeWarning: This filename (c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\data\preprocessed\sub-03_preprocessed_raw.

In [63]:
train_dl, val_dl, test_dl = training.get_loaders(train_ds, val_ds, test_ds, num_workers=0)

In [ ]:
importlib.reload(preprocess)
importlib.reload(training)
importlib.reload(diffe)

from src.diffe import DDPM, ConditionalUNet
import torch
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"
in_channels = train_ds[0].shape[0]
n_feat = 128
n_T = 1000
betas = (1e-6, 1e-2)
base_lr, lr = 9e-5, 1.5e-3
step_size = 150

ddpm_model = ConditionalUNet(in_channels=in_channels, n_feat=n_feat).to(device)
ddpm = DDPM(nn_model=ddpm_model, betas=(1e-6, 1e-2), n_T=n_T, device=device).to(
    device
)
trainer = training.DDPMTrainer(
    model=ddpm,
    model_name="test_run"
)
optimizer = optim.RMSprop(ddpm.parameters(), lr=base_lr)
scheduler = optim.lr_scheduler.CyclicLR(
        optimizer=optimizer,
        base_lr=base_lr,
        max_lr=lr,
        step_size_up=step_size,
        mode="exp_range",
        cycle_momentum=False,
        gamma=0.9998,
    )

trainer.train(
    train_loader=train_dl,
    val_loader=val_dl,
    optimizer=optimizer,
    num_epochs=50,
    patience=10,
)

Saving model test_run at epoch 0 to c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\models
Epoch 1/50, Train Loss: 0.5099, Validation Loss: 0.5238
Saving model test_run at epoch 1 to c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\models
Epoch 2/50, Train Loss: 0.4766, Validation Loss: 0.4896
Saving model test_run at epoch 2 to c:\magister\probabilistyczne-modele-grafowe\lab\mind-reader\src\..\models
Epoch 3/50, Train Loss: 0.4674, Validation Loss: 0.4750
